In [1]:
from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:
from haystack.nodes import PDFToTextConverter
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.nodes import FARMReader, ElasticsearchRetriever
from haystack.pipelines import ExtractiveQAPipeline, DocumentSearchPipeline
from haystack.nodes import PreProcessor
from haystack import Document

In [3]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [4]:
doc_dir = "./"
train_filename = "squad_format_thesis/training_dpr.json"
dev_filename = "squad_format_thesis/dev_dpr.json"
test_filename = "squad_format_thesis/test_dpr.json"

query_model = "IIC/dpr-spanish-question_encoder-allqa-base"
passage_model = "IIC/dpr-spanish-passage_encoder-allqa-base"

save_dir = "./models/model_big"

In [5]:
retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=400,
    use_gpu=False
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-question_encoder-allqa-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Automatically detected language from language model name: spanish
INFO - haystack.modeling.model.language_model -  Loaded IIC/dpr-spanish-question_encoder-allqa-base
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find IIC/dpr-spanish-passage_e

In [6]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=test_filename,
    n_epochs=3,
    batch_size=4,
    grad_acc_steps=10,
    save_dir=save_dir,
    evaluate_every=5,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
    optimizer_name = "SGD"
)

INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: squad_format_thesis/training_dpr.json 
INFO - haystack.modeling.data_handler.data_silo -  Got ya 16 parallel workers to convert 101 dictionaries to pytorch datasets (chunksize = 6)...
INFO - haystack.modeling.data_handler.data_silo -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.data_handler.data_silo -  /w\   /w\   /|\  /w\   /|\  /w\   /|\  /|\  /|\  /w\   /|\  /|\  /w\   /w\   /|\  /w\ 
INFO - haystack.modeling.data_handler.data_silo -  /'\   /'\   /'\   / \   /'\   /'\ 